In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from mmf_speckle import *
from __init__ import *

In [3]:
import torch
import matplotlib.pyplot as plt
import h5py
import time

In [5]:
device="cuda"
dtype=torch.float32
isfloat64=False
root="./results/exp2-x"

### Exp2: add nosie

In [6]:
A = get_mmf_speckle_measure_matrix(0,device,dtype)
PM_image = get_pre_measure_img(device, dtype).reshape((-1,1)) 

In [7]:
# add nosie
nosie=True
nosie_sigma_list = [0,0.5,1,1.5,2,5]

In [8]:
for nosie_sigma in nosie_sigma_list:
    # unknow measurement
    A_u = get_mmf_speckle_measure_matrix(25,device,dtype)
    # reconstruction of two unknown images
    names=[
        "GI",
        "Baboon",
        "Peppers",
        "Goldhill",
        "Barbara",
        "Cameraman",
        "Lena"
    ]
    y_u_list = []
    x_u_list = []
    for name in names:
        x_u = get_gi_image(0,device, dtype) if name=="GI" else get_t_image(name,device, dtype)
        y_u = speckle_measure(A_u, x_u.reshape((-1,1)), nosie=nosie, nosie_sigma=nosie_sigma)
        x_u_list.append(x_u)
        y_u_list.append(y_u)
        
    # Iterative Algorithm 2    
    for j in range(len(names)):
        name = names[j]
        f = h5py.File("{}/algo2/{}/{}.h5".format(root,nosie_sigma,name),"a")

        recv_u_idx = j
        x_u = x_u_list[recv_u_idx].reshape((-1,1))

        params_2 = {
            "A": A,
            "PM_image": PM_image,
            "PM_epoch": 20,
            "epoch": 200,
            "speckle_measure": lambda A: speckle_measure(A, x_u, nosie=nosie, nosie_sigma=nosie_sigma),
            "A_recv":None,
            "e_y":None,
            "isfloat64":isfloat64,
            "calc_error": lambda A: A @ x_u
        }

        decay_k = calc_iterative_algo_k(params_2["A"], params_2["PM_image"], x_u) 
        y_u = y_u_list[recv_u_idx]
        (A_recv, k), pm_errors, errors, times = iterative_algo_2(y_u, params_2, verbose=False)
        
        print(errors[-1])
        print("done")
        
        f.create_dataset("A_recv", data=A_recv.cpu().numpy())
        f.create_dataset("y_u", data=y_u.cpu().numpy())

        f.create_dataset("errors", data=np.array(errors))
        f.create_dataset("pm_errors", data=np.array(pm_errors))

        f.create_dataset("decay_k", data=decay_k.cpu().numpy())
        f.create_dataset("times", data=np.array([times]))   

        f.create_dataset("PM_epoch", data=np.array([params_2["PM_epoch"]]))  
        f.create_dataset("epoch", data=np.array([params_2["epoch"]]))   

        f.close()

tensor([[0.2630],
        [0.2627],
        [0.2637],
        ...,
        [0.2635],
        [0.2630],
        [0.2625]], device='cuda:0')
0.10062754
done
tensor([[1.0154],
        [1.0160],
        [1.0149],
        ...,
        [1.0150],
        [1.0149],
        [1.0153]], device='cuda:0')
0.11205996
done
tensor([[0.8198],
        [0.8203],
        [0.8197],
        ...,
        [0.8211],
        [0.8216],
        [0.8204]], device='cuda:0')
0.1527209
done
tensor([[0.8828],
        [0.8815],
        [0.8815],
        ...,
        [0.8817],
        [0.8809],
        [0.8824]], device='cuda:0')
0.15873593
done
tensor([[0.8818],
        [0.8824],
        [0.8832],
        ...,
        [0.8809],
        [0.8820],
        [0.8820]], device='cuda:0')
0.15214014
done
tensor([[0.9318],
        [0.9317],
        [0.9332],
        ...,
        [0.9332],
        [0.9304],
        [0.9307]], device='cuda:0')
0.23129931
done
tensor([[0.9736],
        [0.9730],
        [0.9746],
        ...,
    

In [10]:
for nosie_sigma in nosie_sigma_list:
    # unknow measurement
    A_u = get_mmf_speckle_measure_matrix(25,device,dtype)
    # reconstruction of two unknown images
    names=[
        "GI",
        "Baboon",
        "Peppers",
        "Goldhill",
        "Barbara",
        "Cameraman",
        "Lena"
    ]
    y_u_list = []
    x_u_list = []
    for name in names:
        x_u = get_gi_image(0,device, dtype) if name=="GI" else get_t_image(name,device, dtype)
        y_u = speckle_measure(A_u, x_u.reshape((-1,1)), nosie=nosie, nosie_sigma=nosie_sigma)
        x_u_list.append(x_u)
        y_u_list.append(y_u)
        
    # Iterative Algorithm 2    
    for j in range(len(names)):
        name = names[j]
        f = h5py.File("{}/algo1/{}/{}.h5".format(root,nosie_sigma,name),"a")

        recv_u_idx = j
        x_u = x_u_list[recv_u_idx].reshape((-1,1))

        params_1 = {
            "A": A,
            "PM_image": PM_image,
            "epoch": 200,
            "speckle_measure": lambda A: speckle_measure(A, x_u, nosie=nosie, nosie_sigma=nosie_sigma),
            "A_recv":None,
            "e_y":None,
            "calc_error": lambda A: A @ x_u
        }
        decay_k = calc_iterative_algo_k(params_1["A"], params_1["PM_image"], x_u) 
        y_u = y_u_list[recv_u_idx]
        A_recv, errors = iterative_algo_1(y_u, params_1, verbose=False)
        
        print(errors[-1])
        print("done")
        
        f.create_dataset("A_recv", data=A_recv.cpu().numpy())
        f.create_dataset("y_u", data=y_u.cpu().numpy())
        f.create_dataset("errors", data=np.array(errors))
        f.create_dataset("decay_k", data=decay_k.cpu().numpy())
        f.close()

5.4541015e-05
done
8.59375e-05
done
6.8945315e-05
done
7.255859e-05
done
7.001953e-05
done
7.255859e-05
done
7.333984e-05
done
0.156793
done
0.4205743
done
0.3327883
done
0.35965225
done
0.37271416
done
0.371078
done
0.39424413
done
0.3089345
done
0.846503
done
0.67441046
done
0.7190667
done
0.72590506
done
0.7568427
done
0.7893469
done
0.46452534
done
1.2460418
done
0.99412966
done
1.0974982
done
1.1017966
done
1.129592
done
1.19444
done
0.62690306
done
1.6853281
done
1.4015422
done
1.4425114
done
1.4365135
done
1.5332044
done
1.6432332
done
1.5954485
done
4.1620264
done
3.4378886
done
3.6732059
done
3.7162867
done
3.8825672
done
4.0868144
done
